In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import keras
from keras import layers, optimizers, metrics, activations, losses

import matplotlib.pyplot as plt

In [ ]:
import os

data_path = "./data/transformed/"

if any(["COLAB" in x for x in list(os.environ.keys())]):
    from google.colab import drive
    drive.mount('/content/drive/')
    data_path = r"/content/drive/MyDrive/Colab Notebooks/digit-recognizer/data/transformed/"

In [ ]:
!ls -R {data_path.replace(" ", "\\ ")}

In [ ]:
# Read Preprocessed files
train = np.load(f"{data_path}/train_transformed_20230606_134803.npz")
train_augmented = np.load(f"{data_path}/augmented/train_augmented_20230606_134915.npz")

test_unseen = np.load(f"{data_path}/test_unseen_transformed_20230606_134803.npz")

X_train, y_train = train['X'], train['y']
X_augmented_train, y_augmented_train = train_augmented['X'], train_augmented['y']
X_test_unseen = test_unseen['X']

In [ ]:
# X = np.concatenate([X_train, X_augmented_train])
# y = np.concatenate([y_train, y_augmented_train])

X = X_train
y = y_train, y_augmented_train

In [48]:
from functools import reduce, partial
from typing import Any
from abc import abstractmethod, ABC

class ResNetBlock(ABC):
    def __init__(self, filters, kernel_size, strides, padding):
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.padding = padding

    @abstractmethod
    def _identity(self, inputs, convolved_output):
        raise NotImplementedError
    
    @abstractmethod
    def _conv(self, inputs, activation):
        raise NotImplementedError
    
class IdentityBlock(ResNetBlock):
    def __call__(self, inputs):
        computations = [
            partial(self._conv, activation = True),
            partial(self._conv, activation = False),
            partial(self._identity, inputs),
            layers.Activation('relu')
        ]

        return reduce(lambda inputs, layer: layer(inputs), computations, inputs)
    
    def _identity(self, inputs, convolved_output):
        return layers.Add()([inputs, convolved_output])

    def _conv(self, inputs, activation = True):
        # Perform convolutions
        conv = layers.Conv2D(
            filters=self.filters, 
            kernel_size=self.kernel_size, 
            strides = self.strides, 
            padding = self.padding)(inputs)
        
        # Perform batch normalization
        result = layers.BatchNormalization()(conv)

        # Acivate outputs
        if activation:
            result = layers.Activation('relu')(result)

        # return result
        return result
    
input_layer = keras.Input((24, 24, 1))
# layer_1 = layers.Conv2D(filters=24, kernel_size=(3, 3), activation='relu', padding = 'SAME')(input_layer)
# layer_2 = layers.Conv2D(filters=24, kernel_size=(3, 3), activation='relu', padding = 'SAME')(layer_1)
id_layer = IdentityBlock(64, (3,3), 1, 'same')(input_layer)

model = keras.Model(inputs = input_layer, outputs = id_layer)


model.compile()

model.summary()

Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_41 (InputLayer)          [(None, 24, 24, 1)]  0           []                               
                                                                                                  
 conv2d_34 (Conv2D)             (None, 24, 24, 64)   640         ['input_41[0][0]']               
                                                                                                  
 batch_normalization_13 (BatchN  (None, 24, 24, 64)  256         ['conv2d_34[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_11 (Activation)     (None, 24, 24, 64)   0           ['batch_normalization_13[0

In [ ]:
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(24, 24, 1)),
        layers.Conv2D(filters=32, kernel_size=3, activation = 'sigmoid'),
        layers.BatchNormalization(),
        layers.Conv2D(filters=16, kernel_size=3, activation = 'sigmoid'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Conv2D(filters=8, kernel_size=3, activation = 'sigmoid'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='sigmoid'),
        layers.Dense(10, activation='softmax')
    ]
)

#Compiling our model with appropriate loss function and optimizer
model.compile(loss = 'categorical_crossentropy', optimizer= 'adam', metrics=['categorical_accuracy'])

# Summary of the model
model.summary()

#Fitting the model
history = model.fit(X, y, epochs=10, validation_split=0.2)

In [ ]:
plt.plot(history.history['loss'], label = 'train_loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.legend()
plt.title("Loss Curve")

In [ ]:
plt.plot(history.history['categorical_accuracy'], label = 'train_accuracy')
plt.plot(history.history['val_categorical_accuracy'], label = 'val_accuracy')
plt.legend()
plt.title("Categorical Accuracy Curve")